<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_night_20250825/blob/main/tibame20250921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import json
import urllib.request as req
import bs4 as bs

def download_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)

    metas = html.find_all("span", {"class":"article-meta-value"})
    uid = metas[0]
    board = metas[1]
    title = metas[2]
    posttime = metas[3]


    uid_text = uid.get_text().strip()
    board_text = board.get_text().strip()
    title_text = title.get_text().strip()
    posttime_text = posttime.get_text().strip()

    pushes = html.find_all("div", {"class":"push"})
    push_list = []
    for p in pushes:
        trans = {"推":1, "→":0, "噓":-1}
        pushtag = p.find("span", {"class":"push-tag"})
        pushuid = p.find("span", {"class":"push-userid"})
        pushcontent = p.find("span", {"class":"push-content"})
        pushipdatetime = p.find("span", {"class":"push-ipdatetime"})

        pushtag_text = pushtag.get_text().strip()
        pushtag_score = trans[pushtag_text]
        pushuid_text = pushuid.get_text().strip()
        pushcontent_text = pushcontent.get_text().strip().replace(": ", "")
        pushipdatetime_text = pushipdatetime.get_text().strip()

        push_meta = {
            "score": pushtag_score,
            "uid": pushuid_text,
            "content": pushcontent_text,
            "ipdatetime": pushipdatetime_text
        }
        push_list.append(push_meta)

    # extract: 我都放最後做(因為刪了就刪了)
    maincontent = html.find("div", {"id":"main-content"})
    targets = html.find_all("div", {"class":"article-metaline"})
    for t in targets:
        t.extract()
    targets = html.find_all("div", {"class":"article-metaline-right"})
    for t in targets:
        t.extract()
    targets = html.find_all("div", {"class":"push"})
    for t in targets:
        t.extract()
    maincontent_text = maincontent.get_text()

    # userid/看板名稱/標題/時間  推虛文: 推需>/userid/內容/ipdatetime
    row = {
        "uid": uid_text,
        "board": board_text,
        "title": title_text,
        "time": posttime_text,
        "content": maincontent_text,
        "pushes": push_list
    }

    dn = url.split("/")[-1]
    fn = dn + "/" + "metadata.json"
    print("Save to:", fn)
    # 如果資料夾不存在, 就make起來
    if not os.path.exists(dn):
        os.makedirs(dn)

    with open(fn, "w", encoding="utf-8") as f:
        json.dump(row, f, ensure_ascii=False, indent=4)



```
當你沒有class/id可以篩選的時候
1. 抓包他的區塊
2. 自己寫
```



In [15]:
def download_imgs(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)

    allow_subs = ["jpg", "jpeg", "png", "gif", "webp"]
    links = html.find_all("a")
    for l in links:
        l_href = l["href"]
        l_subname = l_href.split(".")[-1]
        if l_subname.lower() in allow_subs:
            print("download:", l_href)
            r = req.Request(l_href)
            r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0")
            resp = req.urlopen(r)
            content = resp.read()

            dn = url.split("/")[-1]
            fn = dn + "/" + l_href.split("/")[-1]
            print("Save to:", fn)
            # 如果資料夾不存在, 就make起來
            if not os.path.exists(dn):
                os.makedirs(dn)

            with open(fn, "wb") as f:
                f.write(content)
        else:
            print("pass:", l_href)



```
狀態代碼(看頭)

2: 成功
3: 轉址(輸入A, 轉到B)
4: 失敗
404: Not Found
403:
429: Unknown Error

```



In [12]:
imgurl = "https://i.imgur.com/xoqr8vi.jpeg"
r = req.Request(imgurl)
r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0")
resp = req.urlopen(r)
content = resp.read()

# 純文字: open(檔名, "w"/"r", encoding="utf-8")
# 非純文字: open(檔名, "wb"/"rb")
with open("test.png", "wb") as f:
    f.write(content)

In [ ]:
url = "https://www.ptt.cc/bbs/Beauty/index3999.html"
# 走過每個超連結(拿包他的區塊)
# 拿出href, 完整網址->定義好的兩個函式
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

post_divs = html.find_all("div", {"class":"title"})
for div in post_divs:
    post_link = div.find("a")
    # == None: 裡面沒有a, 被刪文了
    if not post_link == None:
        post_link_href = "https://www.ptt.cc" + post_link["href"]
        download_meta(post_link_href)
        download_imgs(post_link_href)
        print(post_link_href)
    else:
        print("被刪文了!!")

In [23]:
import json
import glob
import pandas as pd

table = []
for fn in glob.glob("*/metadata.json"):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
        row = json.loads(content)
        table.append(row)
df = pd.DataFrame(table)
df

,uid,board,title,time,content,pushes
0,StressND (Plmier47kk),Beauty,[正妹] Anna,Sat Sep 20 15:58:37 2025,\nhttps://i.postimg.cc/1XD6LTsB/250918-ANNA-AT...,"[{'score': 1, 'uid': 'sophia810413', 'content'..."
1,airgp32002 (airgp32002),Beauty,[正妹] 夏日女神祭,Sat Sep 20 18:02:42 2025,\nhttps://i.imgur.com/sxsePjQ.jpeg\n\n--\n※ 發信...,"[{'score': 1, 'uid': 'flybirdy', 'content': '等..."
2,yokann (我怕練太壯),Beauty,[正妹] 周西西,Sun Sep 21 00:16:28 2025,\nhttps://i.imgur.com/VyCnqAj.jpeg\nhttps://i....,[]
3,Kjartan (Olafsson),Beauty,[正妹] 青木裕子 平成年代寫真偶像,Sat Sep 20 19:19:53 2025,\n青木裕子 前寫真偶像 1977年生\n\nhttps://images2.imgbox....,"[{'score': 1, 'uid': 'bio5chris', 'content': '..."
4,mefifthfloor (高雄阿甘),Beauty,[正妹] 不錯的泡芙攤車老闆,Sat Sep 20 19:30:08 2025,\nhttps://meee.com.tw/XN8EAFV.JPG\n\nhttps://m...,"[{'score': 0, 'uid': 'Gundam77', 'content': '不..."
5,Kjartan (Olafsson),Beauty,[正妹] 瑜珈課 軟Q,Sun Sep 21 10:36:41 2025,\nhttps://images2.imgbox.com/91/dd/xLcNofo8_o....,"[{'score': 1, 'uid': 'johnwu', 'content': '木瓜奶..."
6,cryindevil (燈火闌珊處),Beauty,[神人] 9/19桃園機場1航兇狠女神,Sat Sep 20 20:32:30 2025,\n請幫神昨天目擊的女神一枚\n2025/09/19上午7:15\n在桃園機場一航廈出境海關...,"[{'score': -1, 'uid': 'zx246800tw', 'content':..."
7,Gentlemon (肥宅紳士),Beauty,[正妹] Cosplay 2274 韓國 海夢 山田涼 米哈拉,Sat Sep 20 22:13:11 2025,\n\n\n\n\n1.https://i.urusai.cc/BlI4e.jpg\n2.h...,[]
8,FWMK2RC6RS (BIGTITCR),Beauty,"[正妹] 義大利的公關, 專注於高級風格",Sat Sep 20 20:45:10 2025,\n\nIG:https://www.instagram.com/llucelombardi...,"[{'score': 1, 'uid': 'LoserLee', 'content': '極..."
9,FWMK2RC6RS (BIGTITCR),Beauty,"[正妹] 兇猛, 洋裝很合身",Sat Sep 20 21:08:14 2025,\nhttps://images2.imgbox.com/28/bc/mp5ukYfR_o....,"[{'score': 1, 'uid': 'bobbytsi', 'content': '超..."




```
型態:
一個型態可以擁有什麼
1. 專屬功能: str.replace()
2. 專屬值: 人.身高
```



In [25]:
# 內建
import urllib.request as req
# 第三方
import requests

# 純文字對照
# req.urlopen
url = "https://www.ptt.cc/bbs/Beauty/index3999.html"
resp = req.urlopen(url)
content = resp.read()

# requests
url = "https://www.ptt.cc/bbs/Beauty/index3999.html"
resp = requests.get(url)
# .read()->.text
content = resp.text
# 使用第三方函式庫的原因: 有一些比較方便的使用方式
print(resp.request.headers)

{'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [29]:
# 非純文字
imgurl = "https://i.imgur.com/xoqr8vi.jpeg"
r = req.Request(imgurl)
r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0")
resp = req.urlopen(r)
content = resp.read()

# stream=True resp.raw: 非純文字
h = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"
}
resp = requests.get(imgurl, stream=True, headers=h)
# .read() -> .raw.read()
content = resp.raw.read()
print(content)

b'\xff\xd8\xff\xdb\x00\x84\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x07\x80\x048\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x01\xa2\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\x

In [ ]:
import os
import time
import urllib.request as req
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc


# 如果你是MAC, 遇到SSL Certificate Failed: 就加這兩行
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 你要把你所有的平常動作模擬在這裡
if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)
    driver.get("https://www.google.com")
    driver.maximize_window()
    # 找到區塊再做操作
    # bs: find find_all
    # selenium: find_element find_elements
    e = driver.find_element(By.CLASS_NAME, "gLFyf")
    # 最常做的兩個: send_keys鍵盤輸入  click滑鼠點擊
    e.send_keys("chiikawa")
    e.send_keys(Keys.ENTER)
    time.sleep(2)
    # 以下的操作
    e = driver.find_elements(By.CLASS_NAME, "C6AK7c")[1]
    e.click()
    time.sleep(2)
    imgs = driver.find_elements(By.CLASS_NAME, "YQ4gaf")
    for img in imgs:
        # bs: get_text()或.text ["href"]
        # selenium: .text .get_attribute("href")
        src = img.get_attribute("src")
        # urlopen -> urlretrieve
        dn = "google"
        fn = dn + "/" + str(time.time()) + ".jpg"
        if not os.path.exists(dn):
            os.makedirs(dn)
        req.urlretrieve(src, fn)
    time.sleep(3)

In [ ]:
# cookie: 伺服器在回覆的時候會順便要求你設置一組值在你瀏覽器
# 下次你拜訪同樣網頁的時候你就可以把這個值送出, 伺服器就可以檢查了
import os
import time
import urllib.request as req
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc


# 如果你是MAC, 遇到SSL Certificate Failed: 就加這兩行
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 你要把你所有的平常動作模擬在這裡
if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)
    driver.get("https://www.facebook.com")
    driver.maximize_window()

    cookie_s = "自己去拿"
    cookies = cookie_s.split(";")
    for cookie in cookies:
        cookie = cookie.strip()
        key, val = cookie.split("=", 1)
        driver.add_cookie({"name": key, "value": val})

    driver.refresh()

    time.sleep(3)